# Analisis de sentimientos

Creacion de un analizador de sentimientos a partir de texto, usando el conjunto de datos https://ai.stanford.edu/~amaas/data/sentiment/. Este conjunto consta de 2 carpetas, train y test, cada una de las cuales poseen a su vez dos carpetas, pos y neg, las que almacenan comentarios de peliculas, segun recuerdo, positivos y negativos respectivamente. La idea es entrenar un modelo que sea capaz de predecir si los comentarios son positivos o negativos. Teniendo en cuenta que las categorias de los comentarios son positivos o negativos, se hace uso de regresion logistica, la cual a su vez hace uso de la funcion sigmoide. La funcion sigmoide, segun recuerdo, tiene la forma f(x) = 1/ [1 + e^(-t) ], donde t en este caso sera el vector de palabras.


In [ ]:
#Importamos las bibliotecas basicas
import os
import nltk
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [ ]:
#Definimos la carpeta donde estan los conjuntos de entrenamiento y prueba respectivamente
train_folder, test_folder = './aclImdb/train/', './aclImdb/test/'

In [ ]:
"""
Creamos el conjunto de datos. Es decir, para el conjunto de entrenamiento, tomaremos todos los comentarios positivos y los
almacenaremos juntos y etiquetaremos como 1, por otra parte con los comentarios negativos igualmente los almacenaremos en
donde mismo pusimos los comentarios positivos del conjunto de entrenamiento, pero en este caso los etiquetaremos como 0.
"""
def creatingDataSet(folder, folder_type = ['pos/', 'neg/'], result_file_path = None):
    data = []
    
    for f_type in folder_type:
        temp_folder=folder + f_type
        print(temp_folder)
        folder_class = 1
        if f_type != 'pos/':
            folder_class = 0
        #Listando los archivos
        files = os.listdir(temp_folder)
        #Analizando los archivos
        for file in files:
            with open(temp_folder+file, "r", encoding="utf8") as f:
                value = f.read()
                data.append( [value, folder_class] )
    
    df = pd.DataFrame( data = data, columns = ['text', 'class'])
    if result_file_path:
        df.to_csv(result_file_path, index = False)
    return df


In [ ]:
#Creando archivos CSV donde se unifican los datos de entrenamiento, asi como con los datos de prueba
creatingDataSet('./aclImdb/train/', result_file_path='./train.csv')
creatingDataSet('./aclImdb/test/', result_file_path='./test.csv')

#  Vectorizando los conjuntos de entrenamiento y prueba



En esta fase convertiremos cada uno de los textos a vector, es decir a numeros, para su posterior analisis.


In [ ]:


train_dataset = pd.read_csv('./train.csv')
test_dataset = pd.read_csv('./test.csv')
test_dataset



In [ ]:


#Importamos la biblioteca que utilizaremos para la vectorizacion de las palabras
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:


"""
Descartamos las palabras cuya frecuencia es menor a .1 o superior a .5; es decir, descartamos las palabras con una baja
frecuencia, pues estas suelen ser errores ortograficos. Asimismo descartamos las palabras con una alta frecuencia, pues 
estas suelen ser preposiciones, articulos y conjunciones.
"""
tfidf = TfidfVectorizer(min_df = .1, max_df = .5, ngram_range=(1, 3), stop_words = 'english')



In [ ]:


#Obteniendo los textos de entrenamiento y prueba a vectorizar
train_dataset_texts = train_dataset['text'].values
test_dataset_texts = test_dataset['text'].values



In [ ]:


#Transformando los textos de entrenamiento
features = tfidf.fit_transform(train_dataset_texts)
#Ahora el text 
print(train_dataset_texts[0])
#se traduce en
print(features.todense()[0])



In [ ]:
#Obteniendo las clases del conjunto de entrenamiento
train_dataset_class = train_dataset['class'].values
#Transformando las clases del conjunto de entrenamiento a vector columna
train_dataset_class = train_dataset_class.reshape( -1, 1)
print(train_dataset_class.shape)
#Transformando la matriz dispersa a matriz
train_mtx = features.todense()
print(train_mtx.shape)
train_mtx_array = np.array( train_mtx )
train_mtx_array = np.hstack( (train_mtx_array, train_dataset_class) )
print(train_mtx_array.shape)

#Exportando el dataframe
columns = tfidf.get_feature_names_out()
columns = columns.tolist()
columns.append('class')

df = pd.DataFrame( data = train_mtx_array, columns = columns)
df.to_csv('train_vector.csv', index = False)

In [ ]:


features = tfidf.transform(test_dataset_texts)



In [ ]:
#Obteniendo las clases del conjunto de prueba
test_dataset_class = test_dataset['class'].values
#Transformando las clases del conjunto de prueba a vector columna
test_dataset_class = test_dataset_class.reshape( -1, 1)
print(test_dataset_class.shape)
#Transformando la matriz dispersa a matriz
test_mtx = features.todense()
print(test_mtx.shape)
test_mtx_array = np.array( test_mtx )
test_mtx_array = np.hstack( (test_mtx_array, test_dataset_class) )
print(test_mtx_array.shape)

#Exportando el dataframe
columns = tfidf.get_feature_names_out()
columns = columns.tolist()
columns.append('class')

df = pd.DataFrame( data = test_mtx_array, columns = columns)
df.to_csv('test_vector.csv', index = False)


 # Logistic



Una vez los textos estan vectorizados paramos a la creacion del modelo


In [ ]:


#Leemos los conjuntos de datos de entrenamiento y prueba
train_dataset = pd.read_csv('./train_vector.csv')
test_dataset = pd.read_csv('./test_vector.csv')



In [ ]:
#Obtenemos los vectores de los comentarios, es decir, se ignora la ultima columna pues esta indica si el comentario fue
#positivo o negativo
X_train = train_dataset.iloc[:,:-1]
print(X_train.shape)
#Obtenemos la etiqueta de los comentarios
y_train = train_dataset['class'].values
print(y_train)

X_test = test_dataset.iloc[:,:-1]
print(X_test.shape)
y_test = test_dataset['class'].values
print(y_test)

In [ ]:
#Creamos el modelo de regresion logistica
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
#Evaluamos la exactirud del modelo para etiquetar los comentarios presentes en el conjunto de entrenamiento
print('Train Score', clf.score(X_train, y_train))
#Evaluamos la exactirud del modelo para etiquetar los comentarios presentes en el conjunto de prueba
print('Test Score', clf.score(X_test, y_test))




Como vemos tenemos cerca de un 70 porciento de exactitud al etiquetar los comentarios.


# Hashing



Sin embargo, la anterior estrategia de vectorizacion no resulta del todo adecuada cuando trabajamos con conjuntos de datos demasiado extensos, en el orden de los terabytes, es decir, cuando el numero de caracteristicas es demasiado grande. En este momento una mejor estrategia es vectorizar las caracteristicas, de forma tal que para cada palabra cada caracter es multiplicado por un determinado numero primo elevado a la i-esima potencia, para i=0...no_caracteres - 1. Es decir, el hash de una cadena cualquiera es: hash(s) = s[0] + s[1] p^{1} + ... + s[n] p^{n-1}. Ahora vemos un ejemplo de lo anteriormente expuesto:


In [ ]:


from sklearn.feature_extraction.text import HashingVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]
vectorizer = HashingVectorizer(n_features=2**4)
X = vectorizer.fit_transform(corpus)
print(X.shape)
X.todense()





Creamos un vectorizador, el cual aplicara una funcion hash a cada palabra, ademas esta tabla hash constara de 1024 elementos, lo que si bien pudiese provocar que existan colisiones y disminuya la exactitud del modelo, en este caso no es asi. Aunque debemos recordar que en caso de que lo deseemos podemos aumentar el numero de caracteristicas hasta 2^20, lo que disminuye la posibilidad de que ocurran colisiones.


In [ ]:


vectorizer = HashingVectorizer(n_features=2**10, stop_words = 'english')
features = vectorizer.fit_transform(train_dataset_texts)



In [ ]:


#Obteniendo las clases del conjunto de entrenamiento
train_dataset_class = train_dataset['class'].values
#Transformando la matriz dispersa a matriz
train_mtx = features.todense()
print(train_mtx.shape)
train_mtx_array = np.array( train_mtx )

#Las 1024 caracteristicas que surgen son resultados de la aplicacion de una funcion hash, por lo que no se puede hacer el 
#proceso inverso, es decir, a partir de valor hash recuperar la caracteristica a la que esta asociada, pues en realidad 
#no es del todo asi. Motivo por el cual no se exporto los dataframe que resultaron de la vectorizacion.



In [ ]:


features = vectorizer.transform(test_dataset_texts)



In [ ]:


#Obteniendo las clases del conjunto de prueba
test_dataset_class = test_dataset['class'].values
#Transformando las clases del conjunto de prueba a vector columna
test_dataset_class = test_dataset_class.reshape( -1, 1)
print(test_dataset_class.shape)
#Transformando la matriz dispersa a matriz
test_mtx = features.todense()
print(test_mtx.shape)
test_mtx_array = np.array( test_mtx )



In [ ]:
X_train = train_mtx_array
y_train = train_dataset_class

X_test = test_mtx_array
y_test = test_dataset_class


In [ ]:
#Creamos el modelo de regresion logistica
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
#Evaluamos la exactirud del modelo para etiquetar los comentarios presentes en el conjunto de entrenamiento
print('Train Score', clf.score(X_train, y_train))
#Evaluamos la exactirud del modelo para etiquetar los comentarios presentes en el conjunto de prueba
print('Test Score', clf.score(X_test, y_test))